In [19]:
from nixtla import NixtlaClient
import pandas as pd

In [20]:
nixtla_client = NixtlaClient(
    api_key = 'nixtla-tok-WgE8MEuLXfee5pzkScwGhD0FkGhMcM2TF9HlmyBdsYqkRdznaQHgMyd1t4PIWfo8TLhZkF9NRKm65GKL'
)
nixtla_client.validate_api_key()

INFO:nixtla.nixtla_client:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

In [21]:
df = pd.read_csv('./data/5_2022_1_2022_12.csv')
eval_df = pd.read_csv('./data/5_2022_1_2022_12.csv')
df = df[(df['iu_ac'] == 5) & (df['t_1h'] >= '2022-01-01') & (df['t_1h'] <= '2022-11-31')]
df.head()

,iu_ac,t_1h,etat_barre,q
0,5,2022-01-01 00:00:00,3,1068.0
1,5,2022-01-01 01:00:00,3,1462.0
2,5,2022-01-01 02:00:00,3,1467.0
3,5,2022-01-01 03:00:00,3,1505.0
4,5,2022-01-01 04:00:00,3,1359.0


In [24]:
def convert_TimeGPT_to_q(timegpt_fcst_df):
    timegpt_fcst_df = timegpt_fcst_df.rename(columns={'TimeGPT': 'q'})
    print(timegpt_fcst_df.shape)
    return timegpt_fcst_df


In [38]:
timegpt_fcst_df = nixtla_client.forecast(
    df=df, 
    # h=31*24,
    h = 31*24,
    finetune_steps=100,
    finetune_loss='mae',   
    freq='h', 
    time_col='t_1h', 
    target_col='q', 
    model='timegpt-1-long-horizon'
)
timegpt_fcst_df.head()

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


,t_1h,TimeGPT
0,2022-12-01 00:00:00,1644.328003
1,2022-12-01 01:00:00,1403.700684
2,2022-12-01 02:00:00,1028.282104
3,2022-12-01 03:00:00,722.328735
4,2022-12-01 04:00:00,451.141022


In [32]:
# timegpt_fcst_df = nixtla_client.forecast(
#     df=df, 
#     h=7*24, 
#     freq='H', 
#     time_col='t_1h', 
#     target_col='q', 
#     model='timegpt-1-long-horizon')
# timegpt_fcst_df.head()

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
/media/ssd4t/anaconda3/envs/traffic/lib/python3.9/site-packages/utilsforecast/preprocessing.py:126: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = pd.tseries.frequencies.to_offset(freq)
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


,t_1h,TimeGPT
0,2022-12-01 00:00:00,1585.660889
1,2022-12-01 01:00:00,1312.476074
2,2022-12-01 02:00:00,966.434937
3,2022-12-01 03:00:00,699.152832
4,2022-12-01 04:00:00,492.858063


In [39]:
from sklearn.metrics import mean_absolute_error

def mae_cal(timegpt_fcst_df, eval_df):
    timegpt_fcst_df = convert_TimeGPT_to_q(timegpt_fcst_df)
    # 将 t_1h 列转换为日期时间格式
    timegpt_fcst_df['t_1h'] = pd.to_datetime(timegpt_fcst_df['t_1h'])
    eval_df['t_1h'] = pd.to_datetime(eval_df['t_1h'])
    # 合并两个数据框，确保在 t_1h 列上对齐
    merged_df = pd.merge(timegpt_fcst_df, eval_df, on='t_1h', suffixes=('_timegpt', '_eval'))
    # print(merged_df.head())
    # 计算 MAE
    mae = mean_absolute_error(merged_df['q_eval'], merged_df['q_timegpt'])
    print(f"Mean Absolute Error (MAE): {mae}")
    
# nixtla_client.plot(df, timegpt_fcst_df, time_col='t_1h', target_col='q')
mae_cal(timegpt_fcst_df, eval_df)

(744, 2)
Mean Absolute Error (MAE): 110.85792295394405
